In [ ]:
# LSW Mu;i Labelled Classification
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import OneHotEncoder
import scipy.sparse

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from collections.abc import Sequence, Set

dataset_dir_path = "~/Desktop/Tickets"
categorical_columns = ['Kategorie ID', 'Unterkategorie ID']

tickets = pd.read_csv(f"{dataset_dir_path}/tickets.csv")

tickets.head()
# tickets.count()

In [ ]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

def encode_data_BERT(sample_table, x_category_col_keys, y_col_key, test_size = 0.2):
    x_col_keys = ['stemmed_text', 'stemmed_beschreibung'] + x_category_col_keys

    if 0 < test_size:
        x_train_raw, x_test_raw, y_train, y_test = train_test_split(
            sample_table[x_col_keys], # x
            sample_table[y_col_key].to_numpy(), # y
            test_size = test_size,
            random_state = 1)
    else:
        x_train_raw = sample_table[x_col_keys] # x
        x_test_raw = x_train_raw
        y_train = sample_table[y_col_key].to_numpy()
        y_test = y_train


    # 2. Calculate embeddings by calling model.encode()
    train_text_embeddings = model.encode(x_train_raw['stemmed_text'].values.astype('U'))
    train_description_embedding = model.encode(x_train_raw['stemmed_beschreibung'].values.astype('U'))
    combined_train = hstack((train_text_embeddings, train_description_embedding))

    test_text_embeddings = model.encode(x_test_raw['stemmed_text'].values.astype('U'))
    test_description_embedding = model.encode(x_test_raw['stemmed_beschreibung'].values.astype('U'))
    combined_test = hstack((test_text_embeddings, test_description_embedding))


    encoder = OneHotEncoder(handle_unknown = 'ignore')
    encoded_data_train = encoder.fit_transform(x_train_raw[categorical_columns]).toarray()
    encoded_data_test = encoder.transform(x_test_raw[categorical_columns]).toarray()

    # Convert the one-hot encoded arrays to sparse matrices
    encoded_sparse_train = scipy.sparse.csr_matrix(encoded_data_train)
    encoded_sparse_test = scipy.sparse.csr_matrix(encoded_data_test)

    # Combine TF-IDF matrix and one hot encoded matrix horizontally for both training and testing sets
    x_train = hstack((combined_train, encoded_sparse_train))
    x_test = hstack((combined_test, encoded_sparse_test))

    return x_train, x_test, y_train, y_test


